In [1]:
import env
import os
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

# Acquire and Summarize



# 1
- Acquire data from the cloud database.



In [2]:
def get_connection(
    db: str, user: str = env.user, host: str = env.host, password=env.password
) -> str:
    return f"mysql+pymysql://{user}:{password}@{host}/{db}"

In [3]:
def get_zillow_data(file_name="zillow.csv") -> pd.DataFrame:
    if os.path.isfile(file_name):
        return pd.read_csv(file_name)
    query = """select *
               from properties_2017
               left join predictions_2017
               using (id)
               left join airconditioningtype
               using (airconditioningtypeid)
               left join architecturalstyletype
               using (architecturalstyletypeid)
               left join buildingclasstype
               using (buildingclasstypeid)
               left join heatingorsystemtype
               using (heatingorsystemtypeid)
               left join propertylandusetype
               using (propertylandusetypeid)
               left join storytype
               using (storytypeid)
               left join typeconstructiontype
               using (typeconstructiontypeid)
               WHERE latitude IS NOT NULL
               and longitude IS NOT NULL;
            """
    connection = get_connection("zillow")
    df = pd.read_sql(query, connection)
    df.to_csv(file_name, index=False)
    return df

In [4]:
df = get_zillow_data()
df.head(3)

/var/folders/6d/_03p2v5j0rd067n1f34klb7w0000gn/T/ipykernel_6964/3331471128.py:3: DtypeWarning: Columns (61,63,64,67,68) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(file_name)


,typeconstructiontypeid,storytypeid,propertylandusetypeid,heatingorsystemtypeid,buildingclasstypeid,architecturalstyletypeid,airconditioningtypeid,id,parcelid,basementsqft,...,parcelid.1,logerror,transactiondate,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,propertylandusedesc,storydesc,typeconstructiondesc
0,NaN,NaN,269.0,NaN,NaN,NaN,NaN,0,10754147,NaN,...,14297519.0,0.025595,2017-01-01,NaN,NaN,NaN,NaN,Planned Unit Development,NaN,NaN
1,NaN,NaN,261.0,NaN,NaN,NaN,NaN,1,10759547,NaN,...,17052889.0,0.055619,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN
2,NaN,NaN,47.0,NaN,5.0,NaN,NaN,2,10843547,NaN,...,14186244.0,0.005383,2017-01-01,NaN,NaN,Specialized buildings that do not fit in any o...,NaN,Store/Office (Mixed Use),NaN,NaN


In [ ]:
# note logerror: is the previous model prediction (shows how good or bad it was)

# 2
- Summarize your data (summary stats, info, dtypes, shape, distributions, value_counts, etc.)

In [59]:
# still have duplicates 

# 2982285 - total rows

0

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2982285 entries, 0 to 2982284
Data columns (total 69 columns):
 #   Column                        Dtype  
---  ------                        -----  
 0   typeconstructiontypeid        float64
 1   storytypeid                   float64
 2   propertylandusetypeid         float64
 3   heatingorsystemtypeid         float64
 4   buildingclasstypeid           float64
 5   architecturalstyletypeid      float64
 6   airconditioningtypeid         float64
 7   id                            int64  
 8   parcelid                      int64  
 9   basementsqft                  float64
 10  bathroomcnt                   float64
 11  bedroomcnt                    float64
 12  buildingqualitytypeid         float64
 13  calculatedbathnbr             float64
 14  decktypeid                    float64
 15  finishedfloor1squarefeet      float64
 16  calculatedfinishedsquarefeet  float64
 17  finishedsquarefeet12          float64
 18  finishedsquarefeet13  

In [6]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
typeconstructiontypeid,6746.0,5.999555e+00,3.840787e-01,4.000000e+00,6.000000e+00,6.000000e+00,6.000000e+00,1.300000e+01
storytypeid,1623.0,7.000000e+00,0.000000e+00,7.000000e+00,7.000000e+00,7.000000e+00,7.000000e+00,7.000000e+00
propertylandusetypeid,2982285.0,2.600508e+02,1.593121e+01,3.100000e+01,2.610000e+02,2.610000e+02,2.610000e+02,2.790000e+02
heatingorsystemtypeid,1869164.0,4.083750e+00,3.286780e+00,1.000000e+00,2.000000e+00,2.000000e+00,7.000000e+00,2.400000e+01
buildingclasstypeid,12731.0,3.734742e+00,5.003709e-01,1.000000e+00,3.000000e+00,4.000000e+00,4.000000e+00,5.000000e+00
architecturalstyletypeid,6061.0,7.202607e+00,2.436290e+00,2.000000e+00,7.000000e+00,7.000000e+00,7.000000e+00,2.700000e+01
airconditioningtypeid,815362.0,1.945723e+00,3.160507e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.300000e+01
id,2982285.0,1.491142e+06,8.609117e+05,0.000000e+00,7.455710e+05,1.491142e+06,2.236713e+06,2.982284e+06
parcelid,2982285.0,1.332491e+07,7.897112e+06,1.071172e+07,1.164346e+07,1.254525e+07,1.409806e+07,1.696019e+08
basementsqft,1627.0,6.472207e+02,5.387871e+02,2.000000e+01,2.720000e+02,5.350000e+02,8.475000e+02,8.516000e+03


In [7]:
df.dtypes

typeconstructiontypeid    float64
storytypeid               float64
propertylandusetypeid     float64
heatingorsystemtypeid     float64
buildingclasstypeid       float64
                           ...   
buildingclassdesc          object
heatingorsystemdesc        object
propertylandusedesc        object
storydesc                  object
typeconstructiondesc       object
Length: 69, dtype: object

In [8]:
df.shape

(2982285, 69)

In [9]:
df.typeconstructiondesc.value_counts()

typeconstructiondesc
Frame           6669
Concrete          59
Masonry           14
Metal              3
Manufactured       1
Name: count, dtype: int64

In [10]:
def summarize(df, k=1.5) -> None:
    '''
    Summarize will take in a pandas DataFrame
    and print summary statistics:
    
    info
    shape
    outliers
    description
    missing data stats
    
    return: None (prints to console)
    '''
    # print info on the df
    print('Shape of Data: ')
    print(df.shape)
    print('======================\n======================')
    print('Info: ')
    print(df.info())
    print('======================\n======================')
    print('Descriptions:')
    # print the description of the df, transpose, output markdown
    print(df.describe().T.to_markdown())
    print('======================\n======================')
    # lets do that for categorical info as well
    # we will use select_dtypes to look at just Objects
    print(df.select_dtypes('O').describe().T.to_markdown())
    print('======================\n======================')
    print('missing values:')
    print('by column:')
  
    

In [11]:
summarize(df)

Shape of Data: 
(2982285, 69)
Info: 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2982285 entries, 0 to 2982284
Data columns (total 69 columns):
 #   Column                        Dtype  
---  ------                        -----  
 0   typeconstructiontypeid        float64
 1   storytypeid                   float64
 2   propertylandusetypeid         float64
 3   heatingorsystemtypeid         float64
 4   buildingclasstypeid           float64
 5   architecturalstyletypeid      float64
 6   airconditioningtypeid         float64
 7   id                            int64  
 8   parcelid                      int64  
 9   basementsqft                  float64
 10  bathroomcnt                   float64
 11  bedroomcnt                    float64
 12  buildingqualitytypeid         float64
 13  calculatedbathnbr             float64
 14  decktypeid                    float64
 15  finishedfloor1squarefeet      float64
 16  calculatedfinishedsquarefeet  float64
 17  finishedsquarefeet12        

# 3 
- Write a function that takes in a dataframe of observations and attributes and returns a dataframe where each row is an atttribute name, the first column is the number of rows with missing values for that attribute, and the second column is percent of total rows that have missing values for that attribute. 
- Run the function and document takeaways from this on how you want to handle missing values.

In [12]:
df.isnull().sum()

typeconstructiontypeid    2975539
storytypeid               2980662
propertylandusetypeid           0
heatingorsystemtypeid     1113121
buildingclasstypeid       2969554
                           ...   
buildingclassdesc         2969554
heatingorsystemdesc       1114463
propertylandusedesc             0
storydesc                 2980662
typeconstructiondesc      2975539
Length: 69, dtype: int64

In [13]:
df.isnull().mean()

typeconstructiontypeid    0.997738
storytypeid               0.999456
propertylandusetypeid     0.000000
heatingorsystemtypeid     0.373244
buildingclasstypeid       0.995731
                            ...   
buildingclassdesc         0.995731
heatingorsystemdesc       0.373694
propertylandusedesc       0.000000
storydesc                 0.999456
typeconstructiondesc      0.997738
Length: 69, dtype: float64

In [14]:
null_df = pd.DataFrame(
    {"Num_rows_missing": df.isnull().sum(), "Percent_rows_missing": df.isnull().mean()}
)

null_df

,Num_rows_missing,Percent_rows_missing
typeconstructiontypeid,2975539,0.997738
storytypeid,2980662,0.999456
propertylandusetypeid,0,0.000000
heatingorsystemtypeid,1113121,0.373244
buildingclasstypeid,2969554,0.995731
...,...,...
buildingclassdesc,2969554,0.995731
heatingorsystemdesc,1114463,0.373694
propertylandusedesc,0,0.000000
storydesc,2980662,0.999456


In [34]:
def view_columns_null(df):
    df = pd.DataFrame(
        {
            "num_rows_missing_per_col": df.isnull().sum(axis=0),
            "percent_missing": df.isnull().mean(axis=0)
        }
    )
    return df

In [16]:
def view_rows_null(df):
    df = pd.DataFrame(
        {
            "num_col_missing_per_row": df.isnull().sum(axis=1),
            "percent_missing": df.isnull().mean(axis=1)
        }
    )
    return df


In [17]:
view_rows_null(df)

,num_col_missing_per_row,percent_missing
0,47,0.681159
1,46,0.666667
2,37,0.536232
3,36,0.521739
4,36,0.521739
...,...,...
2982280,40,0.579710
2982281,37,0.536232
2982282,33,0.478261
2982283,34,0.492754


In [18]:
view_columns_null(df)

,num_rows_missing_per_col,percent_missing
typeconstructiontypeid,2975539,0.997738
storytypeid,2980662,0.999456
propertylandusetypeid,0,0.000000
heatingorsystemtypeid,1113121,0.373244
buildingclasstypeid,2969554,0.995731
...,...,...
buildingclassdesc,2969554,0.995731
heatingorsystemdesc,1114463,0.373694
propertylandusedesc,0,0.000000
storydesc,2980662,0.999456


# Prepare

# 1
 - Remove any properties that are likely to be something other than single unit properties. (e.g. no duplexes, no land/lot, ...). 
 - There are multiple ways to estimate that a property is a single unit, and there is not a single "right" answer.



In [19]:
df.propertylandusedesc.value_counts()

propertylandusedesc
Single Family Residential                     2152863
Condominium                                    483789
Duplex (2 Units, Any Combination)              114415
Planned Unit Development                        61559
Mobile Home                                     59344
Quadruplex (4 Units, Any Combination)           40731
Triplex (3 Units, Any Combination)              39977
Commercial/Office/Residential Mixed Used         9487
Cluster Home                                     9421
Store/Office (Mixed Use)                         4379
Residential General                              2996
Cooperative                                      1808
Manufactured, Modular, Prefabricated Homes       1219
Townhouse                                         260
Residential Common Area                            36
Inferred Single Family Residential                  1
Name: count, dtype: int64

In [55]:
df = df[df["propertylandusedesc"] == "Single Family Residential"]

df.propertylandusedesc.nunique()

1

In [56]:
df

,typeconstructiontypeid,storytypeid,propertylandusetypeid,heatingorsystemtypeid,buildingclasstypeid,architecturalstyletypeid,airconditioningtypeid,id,parcelid,basementsqft,...,parcelid.1,logerror,transactiondate,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,propertylandusedesc,storydesc,typeconstructiondesc
1,NaN,NaN,261.0,NaN,NaN,NaN,NaN,1.0,10759547.0,NaN,...,17052889.0,0.055619,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN
15,NaN,NaN,261.0,NaN,NaN,NaN,NaN,15.0,11193347.0,NaN,...,17110996.0,0.008669,2017-01-02,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN
16,NaN,NaN,261.0,NaN,NaN,NaN,NaN,16.0,11215747.0,NaN,...,14375300.0,-0.021896,2017-01-02,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN
17,NaN,NaN,261.0,NaN,NaN,NaN,NaN,17.0,11229347.0,NaN,...,12045625.0,-0.017167,2017-01-02,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN
20,NaN,NaN,261.0,2.0,NaN,NaN,1.0,20.0,11324547.0,NaN,...,12325145.0,0.042463,2017-01-02,Central,NaN,NaN,Central,Single Family Residential,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2982268,NaN,NaN,261.0,NaN,NaN,NaN,NaN,2982268.0,14356230.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN
2982270,NaN,NaN,261.0,NaN,NaN,NaN,NaN,2982270.0,14157725.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN
2982272,NaN,NaN,261.0,NaN,NaN,NaN,NaN,2982272.0,14356183.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN
2982275,NaN,NaN,261.0,NaN,NaN,NaN,NaN,2982275.0,14367624.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN


# 2
- Create a function that will drop rows or columns based on the percent of values that are missing:

In [35]:
column_df = view_columns_null(df)

column_df

,num_rows_missing_per_col,percent_missing
typeconstructiontypeid,2975539,0.997738
storytypeid,2980662,0.999456
propertylandusetypeid,0,0.000000
heatingorsystemtypeid,1113121,0.373244
buildingclasstypeid,2969554,0.995731
...,...,...
buildingclassdesc,2969554,0.995731
heatingorsystemdesc,1114463,0.373694
propertylandusedesc,0,0.000000
storydesc,2980662,0.999456


In [36]:
column_df = column_df[column_df["percent_missing"] <= 0.6]

column_df.index[:] # I want to keep 

Index(['propertylandusetypeid', 'heatingorsystemtypeid', 'id', 'parcelid',
       'bathroomcnt', 'bedroomcnt', 'buildingqualitytypeid',
       'calculatedbathnbr', 'calculatedfinishedsquarefeet',
       'finishedsquarefeet12', 'fips', 'fullbathcnt', 'latitude', 'longitude',
       'lotsizesquarefeet', 'propertycountylandusecode', 'propertyzoningdesc',
       'rawcensustractandblock', 'regionidcity', 'regionidcounty',
       'regionidzip', 'roomcnt', 'unitcnt', 'yearbuilt',
       'structuretaxvaluedollarcnt', 'taxvaluedollarcnt', 'assessmentyear',
       'landtaxvaluedollarcnt', 'taxamount', 'censustractandblock',
       'heatingorsystemdesc', 'propertylandusedesc'],
      dtype='object')

In [40]:
column_df.transpose()

,propertylandusetypeid,heatingorsystemtypeid,id,parcelid,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,calculatedfinishedsquarefeet,finishedsquarefeet12,...,unitcnt,yearbuilt,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,censustractandblock,heatingorsystemdesc,propertylandusedesc
num_rows_missing_per_col,0.0,1.113121e+06,0.0,0.0,25.000000,13.000000,1.040890e+06,114224.000000,42165.000000,261499.000000,...,1.001243e+06,44901.000000,43532.000000,31334.000000,1.000000e+00,56994.000000,19820.000000,72053.00000,1.114463e+06,0.0
percent_missing,0.0,3.732443e-01,0.0,0.0,0.000008,0.000004,3.490243e-01,0.038301,0.014138,0.087684,...,3.357302e-01,0.015056,0.014597,0.010507,3.353134e-07,0.019111,0.006646,0.02416,3.736943e-01,0.0


In [44]:
column_df.transpose().columns

Index(['propertylandusetypeid', 'heatingorsystemtypeid', 'id', 'parcelid',
       'bathroomcnt', 'bedroomcnt', 'buildingqualitytypeid',
       'calculatedbathnbr', 'calculatedfinishedsquarefeet',
       'finishedsquarefeet12', 'fips', 'fullbathcnt', 'latitude', 'longitude',
       'lotsizesquarefeet', 'propertycountylandusecode', 'propertyzoningdesc',
       'rawcensustractandblock', 'regionidcity', 'regionidcounty',
       'regionidzip', 'roomcnt', 'unitcnt', 'yearbuilt',
       'structuretaxvaluedollarcnt', 'taxvaluedollarcnt', 'assessmentyear',
       'landtaxvaluedollarcnt', 'taxamount', 'censustractandblock',
       'heatingorsystemdesc', 'propertylandusedesc'],
      dtype='object')

In [37]:
df.columns 

Index(['typeconstructiontypeid', 'storytypeid', 'propertylandusetypeid',
       'heatingorsystemtypeid', 'buildingclasstypeid',
       'architecturalstyletypeid', 'airconditioningtypeid', 'id', 'parcelid',
       'basementsqft', 'bathroomcnt', 'bedroomcnt', 'buildingqualitytypeid',
       'calculatedbathnbr', 'decktypeid', 'finishedfloor1squarefeet',
       'calculatedfinishedsquarefeet', 'finishedsquarefeet12',
       'finishedsquarefeet13', 'finishedsquarefeet15', 'finishedsquarefeet50',
       'finishedsquarefeet6', 'fips', 'fireplacecnt', 'fullbathcnt',
       'garagecarcnt', 'garagetotalsqft', 'hashottuborspa', 'latitude',
       'longitude', 'lotsizesquarefeet', 'poolcnt', 'poolsizesum',
       'pooltypeid10', 'pooltypeid2', 'pooltypeid7',
       'propertycountylandusecode', 'propertyzoningdesc',
       'rawcensustractandblock', 'regionidcity', 'regionidcounty',
       'regionidneighborhood', 'regionidzip', 'roomcnt', 'threequarterbathnbr',
       'unitcnt', 'yardbuildingsqft17',

In [39]:
df.shape

(2982285, 69)

In [52]:
df = pd.concat([df, column_df.transpose()])

df

,typeconstructiontypeid,storytypeid,propertylandusetypeid,heatingorsystemtypeid,buildingclasstypeid,architecturalstyletypeid,airconditioningtypeid,id,parcelid,basementsqft,...,parcelid.1,logerror,transactiondate,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,propertylandusedesc,storydesc,typeconstructiondesc
0,NaN,NaN,269.0,NaN,NaN,NaN,NaN,0.0,10754147.0,NaN,...,14297519.0,0.025595,2017-01-01,NaN,NaN,NaN,NaN,Planned Unit Development,NaN,NaN
1,NaN,NaN,261.0,NaN,NaN,NaN,NaN,1.0,10759547.0,NaN,...,17052889.0,0.055619,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN
2,NaN,NaN,47.0,NaN,5.0,NaN,NaN,2.0,10843547.0,NaN,...,14186244.0,0.005383,2017-01-01,NaN,NaN,Specialized buildings that do not fit in any o...,NaN,Store/Office (Mixed Use),NaN,NaN
3,NaN,NaN,47.0,NaN,3.0,NaN,NaN,3.0,10859147.0,NaN,...,12177905.0,-0.103410,2017-01-01,NaN,NaN,Buildings having exterior walls built of a non...,NaN,Store/Office (Mixed Use),NaN,NaN
4,NaN,NaN,31.0,NaN,4.0,NaN,NaN,4.0,10879947.0,NaN,...,10887214.0,0.006940,2017-01-01,NaN,NaN,Buildings having wood or wood and steel frames,NaN,Commercial/Office/Residential Mixed Used,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2982292,NaN,NaN,0.0,3.732443e-01,NaN,NaN,NaN,0.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.373694,0.0,NaN,NaN
2982293,NaN,NaN,0.0,1.113121e+06,NaN,NaN,NaN,0.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1114463.0,0.0,NaN,NaN
2982294,NaN,NaN,0.0,3.732443e-01,NaN,NaN,NaN,0.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.373694,0.0,NaN,NaN
num_rows_missing_per_col,NaN,NaN,0.0,1.113121e+06,NaN,NaN,NaN,0.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1114463.0,0.0,NaN,NaN


In [51]:
df_gpby = df.groupby(list(df.columns))


In [26]:
def handel_nulls(df):
    df = df[df.isnull().mean(axis=0) <= 0.6]
    return df

In [27]:
# df = df.isnull().mean(axis=0) <= 0.6
# df


In [28]:
# handel_nulls(df)

# ========================================================================================================

# Mall Customers

# 1
- Acquire data from the customers table in the mall_customers database.


In [29]:
def get_mall_data(file_name="mall.csv") -> pd.DataFrame:
    if os.path.isfile(file_name):
        return pd.read_csv(file_name)
    query = """select * from customers
            """
    connection = get_connection("mall_customers")
    df = pd.read_sql(query, connection)
    df.to_csv(file_name, index=False)
    return df

In [30]:
mall_df = get_mall_data()

mall_df.head()

,customer_id,gender,age,annual_income,spending_score
0,1,Male,19,15,39
1,2,Male,21,15,81
2,3,Female,20,16,6
3,4,Female,23,16,77
4,5,Female,31,17,40


# 2


In [53]:
summarize(mall_df)

Shape of Data: 
(200, 5)
Info: 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   customer_id     200 non-null    int64 
 1   gender          200 non-null    object
 2   age             200 non-null    int64 
 3   annual_income   200 non-null    int64 
 4   spending_score  200 non-null    int64 
dtypes: int64(4), object(1)
memory usage: 7.9+ KB
None
Descriptions:
|                |   count |   mean |     std |   min |   25% |   50% |    75% |   max |
|:---------------|--------:|-------:|--------:|------:|------:|------:|-------:|------:|
| customer_id    |     200 | 100.5  | 57.8792 |     1 | 50.75 | 100.5 | 150.25 |   200 |
| age            |     200 |  38.85 | 13.969  |    18 | 28.75 |  36   |  49    |    70 |
| annual_income  |     200 |  60.56 | 26.2647 |    15 | 41.5  |  61.5 |  78    |   137 |
| spending_score |     200 |  50.2  | 25.8

In [60]:
view_columns_null(mall_df)

,num_rows_missing_per_col,percent_missing
customer_id,0,0.0
gender,0,0.0
age,0,0.0
annual_income,0,0.0
spending_score,0,0.0


# 3 
- Detect outliers using IQR.

In [ ]:
def get_upper_outliers(s, k):
    '''
    Given a series and a cutoff value, k, returns the upper outliers for the
    series.

    The values returned will be either 0 (if the point is not an outlier), or a
    number that indicates how far away from the upper bound the observation is.
    '''
    q1, q3 = s.quantile([.25, .75])
    iqr = q3 - q1
    upper_bound = q3 + k * iqr
    return s.apply(lambda x: max([x - upper_bound, 0]))

def add_upper_outlier_columns(df, k):
    '''
    Add a column with the suffix _outliers for all the numeric columns
    in the given dataframe.
    '''
    # outlier_cols = {col + '_outliers': get_upper_outliers(df[col], k)
    #                 for col in df.select_dtypes('number')}
    # return df.assign(**outlier_cols)

    for col in df.select_dtypes('number'):
        df[col + '_outliers'] = get_upper_outliers(df[col], k)

    return df

add_upper_outlier_columns(df, k=1.5)

df.head()


# 3
- Encode categorical columns using a one hot encoder (pd.get_dummies).


In [62]:
mall_df.head(3)

,customer_id,gender,age,annual_income,spending_score
0,1,Male,19,15,39
1,2,Male,21,15,81
2,3,Female,20,16,6


# 4
- Encode categorical columns using a one hot encoder (pd.get_dummies).


In [64]:
mall_df['gender'] = mall_df['gender'].map({'Female': 0, 'Male': 1})

mall_df.head()

,customer_id,gender,age,annual_income,spending_score
0,1,1,19,15,39
1,2,1,21,15,81
2,3,0,20,16,6
3,4,0,23,16,77
4,5,0,31,17,40


# 5 
- Split data into train, validate, and test.


In [68]:
from sklearn.model_selection import train_test_split

def split_data(df: pd.DataFrame) -> pd.DataFrame: # add to wrangle 
    train_validate, test = train_test_split(df, test_size=.2, random_state=123)
    train, validate = train_test_split(train_validate,
                                       test_size=.3,
                                       random_state=123)
    return train, validate, test

train, validate, test = split_data(mall_df)

print(f' train -> {train.shape}')
print(f' validate -> {validate.shape}')
print(f' test -> {test.shape}')

 train -> (112, 5)
 validate -> (48, 5)
 test -> (40, 5)


# 6
- Handles missing values

In [69]:
view_columns_null(mall_df)

,num_rows_missing_per_col,percent_missing
customer_id,0,0.0
gender,0,0.0
age,0,0.0
annual_income,0,0.0
spending_score,0,0.0


# 7 
- Scaling

In [76]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler, QuantileTransformer 

mms = MinMaxScaler() # creeating the scaler 

def scale_train_data(train, 
               validate, 
               test, 
               cols_to_scale,
               scale,
               return_scaler=False):
    '''
    Scales the 3 data splits. 
    Takes in train, validate, and test data splits and returns their scaled counterparts.
    If return_scalar is True, the scaler object will be returned as well
    '''
    # make copies of our original data so we dont gronk up anything
    train_scaled = train.copy()
    validate_scaled = validate.copy()
    test_scaled = test.copy()
    #     make the thing
    scaler = scale
    #     fit the thing
    scaler.fit(train[cols_to_scale])
    # applying the scaler:
    train_scaled[cols_to_scale] = pd.DataFrame(scaler.transform(train[cols_to_scale]),
                                                  columns=train[cols_to_scale].columns.values).set_index([train.index.values])
                                                  
    validate_scaled[cols_to_scale] = pd.DataFrame(scaler.transform(validate[cols_to_scale]),
                                                  columns=validate[cols_to_scale].columns.values).set_index([validate.index.values])
    
    test_scaled[cols_to_scale] = pd.DataFrame(scaler.transform(test[cols_to_scale]),
                                                 columns=test[cols_to_scale].columns.values).set_index([test.index.values])
                                                 
    if return_scaler:
        return scaler, train_scaled, validate_scaled, test_scaled
    else:
        return train_scaled, validate_scaled, test_scaled


cols_to_scale = mall_df['customer_id', 'age', 'annual_income']

scale_train_data(train, validate, test, cols_to_scale, scale= mms, return_scaler=False)


KeyError: ('customer_id', 'age', 'annual_income')

In [72]:
mall_df.head()

,customer_id,gender,age,annual_income,spending_score
0,1,1,19,15,39
1,2,1,21,15,81
2,3,0,20,16,6
3,4,0,23,16,77
4,5,0,31,17,40
